In [11]:
from ISLP import load_data
import numpy as np

In [2]:
Boston = load_data('Boston')
Boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [3]:
Boston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


In [4]:
# I will be working only with medv column
medv = Boston['medv']
medv.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: medv, dtype: float64

In [5]:
medv.describe()

count    506.000000
mean      22.532806
std        9.197104
min        5.000000
25%       17.025000
50%       21.200000
75%       25.000000
max       50.000000
Name: medv, dtype: float64

In [9]:
# Estimate the population mean of medv
mean = sum(medv)/medv.shape[0]
mean

22.532806324110698

In [17]:
# Estimate of the standard error of the mean
std = np.sqrt(sum([(k-mean)**2 for k in medv]) / (medv.shape[0]))
std_minus_one = np.sqrt(sum([(k-mean)**2 for k in medv]) / (medv.shape[0]-1))
print(f'Standard deviation: {std}')
print(f'Standard deviation using n-1 in the divisor: {std_minus_one}')

print(f'Standard deviation of the mean: {std/np.sqrt(medv.shape[0])}')
print(f'Standard deviation of the mean (with n-1): {std_minus_one/np.sqrt(medv.shape[0])}')

Standard deviation: 9.188011545278206
Standard deviation using n-1 in the divisor: 9.19710408737982
Standard deviation of the mean: 0.4084569346972867
Standard deviation of the mean (with n-1): 0.4088611474975351


In [38]:
def boot_mean(data, n=None, B=10000, seed=0):
    rng = np.random.default_rng(seed)
    n = n or data.shape[0]
    first = 0
    second = 0
    for _ in range(B):
        idx = rng.choice(data.index, n, replace=True)
        # mean of selected indexes
        mean = sum(medv.iloc[idx])/n
        first += mean
        second += mean**2
        #std = np.sqrt(sum([(k-mean)**2 for k in medv.iloc[idx]]) / (n-1))
        #print(mean, std)
    return np.sqrt(second/B - (first/B)**2)

In [44]:
std_boot = boot_mean(medv)

In [45]:
# 95% confidence interval for the mean of medv
[mean - 2*std_boot, mean + 2*std_boot]

[21.71059399457225, 23.355018653649147]

In [47]:
n = medv.shape[0]
[medv.mean() - 2*medv.std()/np.sqrt(n), medv.mean() + 2*medv.std()/np.sqrt(n)]

[21.715084029115605, 23.35052861910575]

In [50]:
# Repeat for the median

print('median with median() method:', medv.median())

def boot_median(data, n=None, B=10000, seed=0):
    rng = np.random.default_rng(seed)
    n = n or data.shape[0]
    first = 0
    second = 0
    for _ in range(B):
        idx = rng.choice(data.index, n, replace=True)
        # median of selected indexes
        median = medv.iloc[idx].median()
        first += median
        second += median**2
        
    return np.sqrt(second/B - (first/B)**2)

std_median_boot = boot_median(medv)
print('median with bootstrap:', std_median_boot)

median with median() method: 21.2
median with bootstrap: 0.37687862766090385


In [55]:
# 10th percentile of medv
np.percentile(medv, 10)

12.75

In [57]:
# 10th percentile using bootstrap
def boot_10_percentile(data, n=None, B=10000, seed=0):
    rng = np.random.default_rng(seed)
    n = n or data.shape[0]
    first = 0
    second = 0
    for _ in range(B):
        idx = rng.choice(data.index, n, replace=True)
        # 10th percentile
        perc_10 = np.percentile(medv.iloc[idx], 10)
        first += perc_10
        second += perc_10**2
        
    return np.sqrt(second/B - (first/B)**2)

print('10th percentile standard error:', boot_10_percentile(medv))

10th percentile standard error: 0.5035641412764847
